In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os
import sys
import lmdb
import pickle
import numpy as np

In [4]:
os.chdir('../') # adress to git dir
print(os.getcwd())

/home/azeghost/git/lmdb_new/LMDB_Datasets


In [5]:
from transformation.lmdb_transformer import LmdbTransformer

In [ ]:
# Store images in lmdb

In [6]:
images_dir = '/home/azeghost/git/Generative_Models/data/.pokemon/' #Folder to images
validation_percentage = 30
valid_format = 'png'

In [7]:
transformer = LmdbTransformer(image_dir = images_dir, 
                              validation_pct = validation_percentage, valid_image_formats = valid_format)

  DEBUG    | Looking for images in 'DS06'
  INFO     | 809 file found


In [8]:
info = {
    'Dataset_name': "Pokemon",
    'Dataset_size': "809",
    'val_num_images':"218",
    'tra_num_images' : '591'
}

transformer.save_metadata('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/meta_info.txt',info)

In [9]:
transformer.get_metadata('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/meta_info.txt')

{'Dataset_name': 'Pokemon',
 'Dataset_size': '809',
 'val_num_images': '218',
 'tra_num_images': '591'}

In [ ]:
from datasets_customed.label_by_filename import get_label_by_filename

In [ ]:
import os
import numpy as np


def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-1:]
    return {'label1': np.array(vid_img_arr), 'label2': np.array(vid_img_arr), 'label3': np.array(vid_img_arr)}

In [ ]:
transformer.transform_store(labels_fn=get_label_by_filename,image_dir=images_dir, lmdb_dir = '.data/Pokemon_LMDB'
           ,category='training',target_size=None,color_mode='rgb')

In [ ]:
transformer.transform_store(labels_fn=get_label_by_filename,image_dir=images_dir, lmdb_dir = '.data/Pokemon_LMDB'
           ,category='validation',target_size=None,color_mode='rgb')

# Read the lmdb 

#Todo
 - Env put outside of loops for write and read
 - Try the Dynamic code again with clean kernel 
 - Create Standartize as an option

In [ ]:
def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], {}
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            dataset = pickle.loads(data)
            images.append(dataset.get_image())
            labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

            for label in labels_list:
                # _lab = {label: eval(f'dataset.{label}')}
                # labels = {**labels, **_lab}
                if label in labels:
                    labels[label].append(eval(f'dataset.{label}'))
                else:
                    labels = {label: [eval(f'dataset.{label}')] }
                
    env.close()
    return {'images': images, **labels}

In [ ]:
ds = read_many_lmdb('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_training', 591)

In [ ]:
ds.keys()

In [ ]:
ds['images'][0].shape

In [ ]:
ds['label1'][0], ds['label2'][0], ds['label3'][0], 

# Zip and upload to git

In [13]:
import os
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_Tar')

FileNotFoundError: [Errno 2] No such file or directory: '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_Tar'

In [12]:
!tar -cvjf pokemon.tar.bz Pokemon_LMDB/* 

Pokemon_LMDB/_training/
Pokemon_LMDB/_training/lock.mdb
Pokemon_LMDB/_training/data.mdb
Pokemon_LMDB/_validation/
Pokemon_LMDB/_validation/lock.mdb
Pokemon_LMDB/_validation/data.mdb
Pokemon_LMDB/meta_info.txt


In [ ]:
!split -b 50M pokemon.tar.bz "pokemon.tar.part"
#split -b <max size> <name of zip or dir to zip/name> <split file name beginning>

In [ ]:
#Check if they are created
!ls -lh pokemon.tar.part*

In [ ]:
!ls -la

In [ ]:
#move to correct folder and push to git
# !mkdir /home/azeghost/git/LMDB_Datasets/pokemon
# !mv pokemon.tar.part* /home/azeghost/git/Generative_Models/data/.pokemon
# !ls -la /home/azeghost/git/Generative_Models/data/.pokemon 

# Combine Them

In [ ]:
#pokemon_training
!cat pokemon.tar.part* > pokemon_combined.tar.bz
#!cat <split files put * at the end> > <final zip name>
!ls -la 
!pwd

In [ ]:
!mkdir ./.test

In [ ]:
!tar -xf pokemon_combined.tar.bz --directory ./.test

In [ ]:
!ls -la ./.test/Pokemon_LMDB

In [ ]:
# !rm -rf ./.test

## LMDBImageIterator and LMDBImageGenerator

In [ ]:
os.chdir('/home/azeghost/git/Generative_Models/')

In [ ]:
os.getcwd()

In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from keras import backend as K
import logging
from utils.reporting.logging import log_message
from utils.data_and_files.file_utils import get_file_path

class LMDBImageIterator(Iterator):


    def __init__(self,
                 num_images,
                 category,
                 lmdb_dir,
                 batch_size,
                 episode_len=20,
                 episode_shift=10,
                 shuffle=True,
                 seed=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='jpeg',
                 dtype=K.floatx(),
                 ):
        
        self.category = category
        self.batch_size = batch_size
        
        
        self.lmdb_dir = lmdb_dir
        self.episode_len = episode_len
        self.episode_shift = episode_shift

        
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        print("Initializing Iterator " + category +" Number of images " +str(num_images))
        print(category,lmdb_dir, batch_size,shuffle,seed)
        self.env = lmdb.open(lmdb_dir, readonly=True)
        
        Iterator.__init__(self, num_images, batch_size, shuffle, seed)
        
        
    def __del__(self):
        self.env.close()
        
        
    def _get_batches_of_transformed_samples(self, index_array):
        print(index_array)
        images, labels = [], []
        
        if len(index_array) < self.batch_size:
            diff = self.batch_size//len(index_array) + 1
            index_array = np.repeat(index_array, diff, axis=0)[:self.batch_size]

        else:
                with self.env.begin() as txn:
                    for image_id in index_array:
                        data = txn.get(f"{image_id:08}".encode("ascii"))
                        dataset = pickle.loads(data)
                        images.append(dataset.get_image())
                        labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

                        for label in labels_list:
                            _lab = {label: eval(f'dataset.{label}')}
                            labels = {**labels, **_lab}
        return {'images': images, **labels}


In [ ]:
import glob
import hashlib
import logging
import os
import re
import warnings

from keras.preprocessing.image import ImageDataGenerator

from utils.data_and_files.data_utils import as_bytes
from utils.reporting.logging import log_message


class LMDBImageGenerator(ImageDataGenerator):
    def flow_from_lmdb_lists(self, 
                              num_images,
                              category,
                              lmdb_dir,
                              batch_size,
                              episode_len=None,
                              episode_shift=None,
                              color_mode='rgb',
                              shuffle =True,
                              seed=None
                              ):

        
          

        return LMDBImageIterator(
                             num_images = num_images,
                             category = category,
                             lmdb_dir = lmdb_dir,
                             batch_size  = batch_size,
                             episode_len = episode_len,
                             episode_shift =episode_shift,
                             shuffle = shuffle,
                             seed = seed)


def get_generators( val_lmdb_dir, val_num_images, tra_lmdb_dir, tra_num_images, 
                   batch_size, episode_len=None, episode_shift=None):

    train_datagen = LMDBImageGenerator()

    valid_datagen = LMDBImageGenerator()

    train_generator = train_datagen.flow_from_lmdb_lists(
        num_images = tra_num_images,
        category='training',
        lmdb_dir=tra_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    validation_generator = valid_datagen.flow_from_lmdb_lists(
        num_images = val_num_images,
        category='validation',
        lmdb_dir=val_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    return train_generator, validation_generator



In [ ]:
training_generator, testing_generator = get_generators(
    val_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_validation',
    val_num_images = 218,
    tra_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_training', 
    tra_num_images = 591,
    batch_size=100
)

In [ ]:
x, y = training_generator.next()

#  Scale test for Pokemon 

In [ ]:
import os
os.chdir('/home/azeghost/git/LMDB_Datasets')

In [ ]:
#Wrapper class for dataset
class Pokemon_Image:
    def __init__(self, image, label):
        self.channels = image.shape[2]
        self.size = image.shape[:2]
        self.image = image.tobytes()
        self.label = label #additional data to be stored (make it string)

    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.float32) #pay attention if you  don't use create_image_lists
        return images.reshape(*self.size, self.channels)     #then dtype will be different

In [ ]:
def store_single_lmdb(filename, img , index, label, num_images):
    """ Stores a wrapper to LMDB.
    """
    map_size = num_images * img.nbytes * 10
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
          # All key-value pairs need to be Strings
          value = Pokemon_Image(img, label)
          key = f"{index:08}"
          txn.put(key.encode("ascii"), pickle.dumps(value))

    env.close()

In [ ]:
def store_lmdb(image_lists, image_dir, 
               lmdb_dir = '/content/LMDB',category='training', target_size=None,
               color_mode='rgb', save_prefix='', save_format='png'):
  classes = list(image_lists.keys())
  num_class = len(classes)
  class2id = dict(zip(classes, range(len(classes))))
  id2class = dict((v, k) for k, v in class2id.items())
 
  y = None
  X = None

  for label_name in classes:
    num_images = len(image_lists[label_name][category])
    print('Storing '+ str(num_images)+lmdb_dir+os.sep+'_{}'.format(category))
    for index, _ in enumerate(image_lists[label_name][category]):
        img_path = get_file_path(image_lists,
                                  label_name,
                                  index,
                                  image_dir,
                                  category)
        img = img_to_array(
                        load_img(
                        img_path,
                        grayscale=color_mode=='grayscale',
                        target_size=target_size
                        )
                    )
        name, _ = os.path.splitext(img_path)
        vid_img_arr = name.split(sep=os.sep)[-1:]
        y = [np.array(vid_img_arr)]     
        name =  lmdb_dir+os.sep+'_{}'.format(category)
        
        store_single_lmdb(index = index, filename=name, img = img,label = y ,num_images = num_images)

In [ ]:
class LMDB_transformer:
    def __init(image_lists**, )
        create_img_list
    
    def transform_store():
        store_lmdb()

In [ ]:
!rm -rf ./LMDB/
!mkdir ./LMDB/
# save_to_dir=None,
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='training',target_size=None,color_mode='rgb',save_prefix='',save_format='png')
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='validation',target_size=None,color_mode='rgb',save_prefix='',save_format='png')

# Done on ubuntu system is local machine does not have tar 
 - Install git bash then u can use such commands.
 - Go to windows store and download Ubuntu emulator wich just installs bash
 - On colab these should work